In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import DecisionTreeRegressor

In [0]:
IS_SPARK_SUBMIT_CLI = True

if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

#LOAD DATA  -- KTSample.csv

In [0]:
file_location = "/FileStore/tables/KTSample.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
temp_table_name1 = "KTSample_csv"

df1.createOrReplaceTempView(temp_table_name1)

In [0]:
if IS_SPARK_SUBMIT_CLI:
    KTSample = spark.read.csv('/FileStore/tables/KTSample.csv', inferSchema=True, header=True)
else:
    KTSample = sqlContext.sql("select * from KTSample_csv")
    
KTSample.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
 1| 2019-11-01 00:30:41| 2019-11-01 00:32:25| 1| 0.0| 1| N| 145| 145| 2| 3.0| 0.5| 0.5| 0.0| 0.0| 0.3| 4.3| 0.0|
 1| 2019-11-01 00:34:01| 2019-11-01 00:34:09| 1| 0.0| 1| N| 145| 145| 2| 2.5| 0.5| 0.5| 0.0| 0.0| 0.3| 3.8| 0.0|
 2| 2019-11-01 00:41:59| 2019-11-01 00:42:23| 1| 0.0| 1| N| 193| 193| 1| 2.5| 0.5| 0.5| 0.95| 0.0| 0.3| 4.75| 0.0|
 2| 2019-11-01 00:02:39| 2019-11-01 00:02:51| 1| 0.0| 1| N| 193| 193| 1| 2.5| 0.5| 0.5| 0.95| 0.0| 0.3| 4.75| 0.0|
 2| 2019-11-01 00:18:30| 2019-11-01 00:18:39| 2| 0.0| 1| N| 226| 226| 2| 2.5| 0.0| 0.5| 0.0| 0.0| 0.3| 3.3| 0.0|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
only showing top 5 rows

#Select Data and Calculate the Trip Time in SEC

In [0]:
timediff=KTSample.select('tpep_pickup_datetime', 'tpep_dropoff_datetime','passenger_count','trip_distance',col('fare_amount').alias('label'))

timediff.show(5)

+--------------------+---------------------+---------------+-------------+-----+
tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|label|
+--------------------+---------------------+---------------+-------------+-----+
 2019-11-01 00:30:41| 2019-11-01 00:32:25| 1| 0.0| 3.0|
 2019-11-01 00:34:01| 2019-11-01 00:34:09| 1| 0.0| 2.5|
 2019-11-01 00:41:59| 2019-11-01 00:42:23| 1| 0.0| 2.5|
 2019-11-01 00:02:39| 2019-11-01 00:02:51| 1| 0.0| 2.5|
 2019-11-01 00:18:30| 2019-11-01 00:18:39| 2| 0.0| 2.5|
+--------------------+---------------------+---------------+-------------+-----+
only showing top 5 rows

In [0]:
df2=timediff.withColumn('tpep_pickup_datetime',to_timestamp(col('tpep_pickup_datetime')))\
  .withColumn('tpep_dropoff_datetime', to_timestamp(col('tpep_dropoff_datetime')))\
  .withColumn('trip_time_in_secs',col("tpep_dropoff_datetime").cast("long") - col('tpep_pickup_datetime').cast("long"))
df2.show(5)

+--------------------+---------------------+---------------+-------------+-----+-----------------+
tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|label|trip_time_in_secs|
+--------------------+---------------------+---------------+-------------+-----+-----------------+
 2019-11-01 00:30:41| 2019-11-01 00:32:25| 1| 0.0| 3.0| 104|
 2019-11-01 00:34:01| 2019-11-01 00:34:09| 1| 0.0| 2.5| 8|
 2019-11-01 00:41:59| 2019-11-01 00:42:23| 1| 0.0| 2.5| 24|
 2019-11-01 00:02:39| 2019-11-01 00:02:51| 1| 0.0| 2.5| 12|
 2019-11-01 00:18:30| 2019-11-01 00:18:39| 2| 0.0| 2.5| 9|
+--------------------+---------------------+---------------+-------------+-----+-----------------+
only showing top 5 rows

In [0]:
data=df2.select('passenger_count','trip_distance','trip_time_in_secs','label')

data.show(5)

+---------------+-------------+-----------------+-----+
passenger_count|trip_distance|trip_time_in_secs|label|
+---------------+-------------+-----------------+-----+
 1| 0.0| 104| 3.0|
 1| 0.0| 8| 2.5|
 1| 0.0| 24| 2.5|
 1| 0.0| 12| 2.5|
 2| 0.0| 9| 2.5|
+---------------+-------------+-----------------+-----+
only showing top 5 rows

#Setup Train and Test datasets

In [0]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

#Setup GBT-Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
gbt = GBTRegressor(labelCol="label")

In [0]:
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 3])\
  .addGrid(gbt.maxIter, [10, 20])\
  .build()
  
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())

cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)

In [0]:
pipeline = Pipeline(stages=[assembler, cv])
pipelineModel = pipeline.fit(train)

/databricks/spark/python/pyspark/ml/util.py:800: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features", "prediction", "trueLabel")
predicted.show(10)

+---------------+------------------+---------+
 features| prediction|trueLabel|
+---------------+------------------+---------+
 [0.0,2.0,0.0]|14.059542516961875| 52.0|
 [0.0,4.0,0.0]|14.059542516961875| 26.0|
 [0.0,9.0,0.0]|14.059542516961875| 52.0|
 [0.0,38.0,0.1]|14.059542516961875| 52.0|
 [0.0,43.0,0.2]|14.059542516961875| 3.0|
 [0.0,3.0,0.3]|14.059542516961875| 6.5|
[0.0,233.0,0.3]| 4.896545420615045| 4.5|
[0.0,271.0,0.3]| 4.896545420615045| 4.5|
[0.0,291.0,0.3]| 4.896545420615045| 5.0|
 [0.0,80.0,0.4]|5.1950118643949414| 3.5|
+---------------+------------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(10)

+---------+------------------+
trueLabel| prediction|
+---------+------------------+
 52.0|14.059542516961875|
 26.0|14.059542516961875|
 52.0|14.059542516961875|
 52.0|14.059542516961875|
 3.0|14.059542516961875|
 6.5|14.059542516961875|
 4.5| 4.896545420615045|
 4.5| 4.896545420615045|
 5.0| 4.896545420615045|
 3.5|5.1950118643949414|
+---------+------------------+
only showing top 10 rows

#RMSE for GBT-Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print( "Root Mean Square Error (RMSE) for GBT Regression :", rmse)

Root Mean Square Error (RMSE) for GBT Regression : 5.333566329311697

#Setup Linear Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)
pipeline1 = Pipeline(stages=[assembler, lr])

In [0]:
paramGrid1 = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
trainval = TrainValidationSplit(estimator=pipeline1, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid1, trainRatio=0.8)

In [0]:
pipelineModel = trainval.fit(train)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features", "prediction", "trueLabel")
predicted.show(10)

+---------------+------------------+---------+
 features| prediction|trueLabel|
+---------------+------------------+---------+
 [0.0,2.0,0.0]|5.1978132523369345| 52.0|
 [0.0,4.0,0.0]| 5.197919995322582| 26.0|
 [0.0,9.0,0.0]| 5.198186852786701| 52.0|
 [0.0,38.0,0.1]| 5.46985492399756| 52.0|
 [0.0,43.0,0.2]| 5.74024207938065| 3.0|
 [0.0,3.0,0.3]|6.0082275175866755| 6.5|
[0.0,233.0,0.3]| 6.020502960936122| 4.5|
[0.0,271.0,0.3]| 6.022531077663421| 4.5|
[0.0,291.0,0.3]| 6.023598507519895| 5.0|
 [0.0,80.0,0.4]|6.2824574204530705| 3.5|
+---------------+------------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(10)

+---------+------------------+
trueLabel| prediction|
+---------+------------------+
 52.0|5.1978132523369345|
 26.0| 5.197919995322582|
 52.0| 5.198186852786701|
 52.0| 5.46985492399756|
 3.0| 5.74024207938065|
 6.5|6.0082275175866755|
 4.5| 6.020502960936122|
 4.5| 6.022531077663421|
 5.0| 6.023598507519895|
 3.5|6.2824574204530705|
+---------+------------------+
only showing top 10 rows

#RMSE for Linear Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print ("Root Mean Square Error (RMSE) for Linear Regression :", rmse)

Root Mean Square Error (RMSE) for Linear Regression : 5.747904934873378

#Decision Forest Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
dt = DecisionTreeRegressor(labelCol="label",featuresCol="features")

In [0]:
paramGrid2 = ParamGridBuilder()\
  .addGrid(dt.maxDepth, [2,3])\
  .addGrid(dt.maxBins, [10,20])\
  .build()

In [0]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol=dt.getLabelCol(), predictionCol=dt.getPredictionCol())

dtcv = CrossValidator(estimator = dt, estimatorParamMaps = paramGrid2, evaluator = evaluator, numFolds=2)

In [0]:
pipeline2 = Pipeline(stages=[assembler, dtcv])
pipelineModel = pipeline2.fit(train)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features","prediction","truelabel")
predicted.show(10)

+---------------+-----------------+---------+
 features| prediction|truelabel|
+---------------+-----------------+---------+
 [0.0,2.0,0.0]|7.023230108963816| 52.0|
 [0.0,4.0,0.0]|7.023230108963816| 26.0|
 [0.0,9.0,0.0]|7.023230108963816| 52.0|
 [0.0,38.0,0.1]|7.023230108963816| 52.0|
 [0.0,43.0,0.2]|7.023230108963816| 3.0|
 [0.0,3.0,0.3]|7.023230108963816| 6.5|
[0.0,233.0,0.3]|7.023230108963816| 4.5|
[0.0,271.0,0.3]|7.023230108963816| 4.5|
[0.0,291.0,0.3]|7.023230108963816| 5.0|
 [0.0,80.0,0.4]|7.023230108963816| 3.5|
+---------------+-----------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(5)

+---------+-----------------+
trueLabel| prediction|
+---------+-----------------+
 52.0|7.023230108963816|
 26.0|7.023230108963816|
 52.0|7.023230108963816|
 52.0|7.023230108963816|
 3.0|7.023230108963816|
+---------+-----------------+
only showing top 5 rows

#RMSE for Decision Forest Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print( "Root Mean Square Error (RMSE) for Decision Forest Regression :", rmse)

Root Mean Square Error (RMSE) for Decision Forest Regression : 5.99980552230505